# Detection of faces in images / 画像中の顔の検出

This is our first cloud service example. We will be using the service to detect faces in a photo.
<br>
Below we explain the code step-by-step.

最初の紹介するクラウドサービスの例です。 このサービスを使って写真の中の顔を検出します。
<br>
以下では、コードを段階的に説明します。

### Authentication / 認証

We need to set the environment variable for authentication, as explained in the previous notebook. 

前のセッションで説明したように、認証のため環境変数を設定する必要があります。

In [ ]:
%env GOOGLE_APPLICATION_CREDENTIALS=gcp_credentials/mlpractice2020.json

### Loading a test photo / テスト写真を読み込む

Let's first take a look at the photo that we will use ("test.jpg"). There are several people in it and we want to detect all their faces. 
<br>
We will need the name of the image file several times so we will put in a variable.

まずは使用する写真「test.jpg」を見てみましょう。写真には何人かの人が写っています。ここからすべての顔を検出しましょう。
<br>
画像ファイルの名前が何回も使いますので、変数に入れます。

In [ ]:
imagefile = "test.jpg"

We will use the OpenCV and Matplotlib libraries to open the picture and plot it.

OpenCVとMatplotlibライブラリを使って写真を読み込んでプロットします。

In [ ]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

I = cv2.imread(imagefile)

# color image loaded by OpenCV is in BGR mode, but Matplotlib displays in RGB mode. 
# this command fixes that
I = I[:,:,[2,1,0]]

# plot the image
plt.imshow(I)
_=plt.axis("off")

### Running GCP face detection / GCP顔検出の実行

First import necessary libraries to work with the Google cloud. 
<br>
(If you use your own local installation, you will need to install the libraries first. You can do it using pip: `pip install google-cloud-vision`)

まずGoogleクラウドを使用するために必要なライブラリをインポートします。
<br>
（自分のPCの環境を使っている場合、最初はライブラリをインストールする必要があります。pipを使えば簡単にインストールすることができます： `pip install google-cloud-vision`）

In [ ]:
from google.cloud import vision
from google.cloud.vision import types

The GCP methods expect the image data to be in the binary format. 
<br>
Load the image like this:

GCPの機能を使うためには画像データがバイナリ形式であることが必要です。
<br>
画像を以下のようにロードします。

In [ ]:
with open(imagefile, "rb") as fd:
    content = fd.read()

Create an `Image` object with the loaded image data.

ロードされた画像データを使って`Image`オブジェクトを作成します。

In [ ]:
image = types.Image(content=content)

Create an ImageAnnotatorClient object.

次はImageAnnotatorClientオブジェクトを作成します。

In [ ]:
client = vision.ImageAnnotatorClient()

Finally, the central part. Call the `face_detection` method.

最後に、中心の部分です。`face_detection`メソッドを呼び出します。

In [ ]:
faces = client.face_detection(image=image).face_annotations

The above method takes some time to execute because it sends the image to a Google server and waits to receive the result. 

上記のメッソドは画像をGoogleサーバーに送信して結果を受け取るのを待つため、実行に時間がかかります。

<div style="background-color:#FFDDDD; padding: 20pt;">

### Cost and quotas / コストとクオータ

_Each time_ you run the above `face_detection` method you are making one call to the Google Cloud and each call is adding to the total cost of using the cloud. Moreover, the Service account key has a quota (limitation on how many calls can be made with this key), to prevent excessive usage by mistake.

Therefore, __avoid calling this method repeatedly__. Instead call it just once for each image you need to process (preferably put the call in a separate cell) and afterwards use the variable with the obtained result.

上の `face_detection`メソッドを _実行するたびに_ 、Google Cloudのサービスを1回呼び出されます。そのたびにクラウドの使用料金が加算されます。さらに、誤って過剰に使用するのを防ぐために、サービスアカウントキーに対してクオータ（使用量の制限の設定）が設定されています。

したがって、このメソッドを __繰り返し呼び出さないでください__ 。代わりに、処理する必要がある画像ごとに1回だけ呼び出してください（できれば、上みたいに、呼び出しを別のセルに入れて）。その後、取得した結果を含む変数を使用できます。

</div>

The `faces` variable contains the whole result. Let's see how many faces were found.

`faces`変数は全体の結果を含みます。顔がいくつ見つかったか見てみましょう。

In [ ]:
print('Number of faces: {}'.format(len(faces)))

`faces` is an array where each element contains the result for one detected face. We will now print the result for the first detected face (`faces[0]`) to see how the full result looks. You will see that it contains a variety of information.
<br>
(The explanation of all the fields in the result can be found [here](https://cloud.google.com/vision/docs/reference/rpc/google.cloud.vision.v1#google.cloud.vision.v1.FaceAnnotation))

`faces`変数は配列です。各要素が一つの検出された顔に対する情報を含みます。結果がどのように見えるかを確認するため、一番目に検出された顔（`faces [0]`）の結果をプリントしましょう。さまざまな情報が含まれていることがわかると思います。
<br>
（結果のすべてのフィールドの説明を見たかったら[こちらをクリックしてください](https://cloud.google.com/vision/docs/reference/rpc/google.cloud.vision.v1#google.cloud.vision.v1.FaceAnnotation)）

In [ ]:
print(faces[0])

### Visualising the result / 結果の視覚化

In the result that we printed above you can see that the first printed field is named `bounding_poly`. It contains the position of the rectangle that surrounds the detected face. It has 4 subfields named `vertices`, one for each corner of the rectangle. The order of the `vertices` (corners) is top-left, top-right, bottom-right, bottom-left. Each of the `vertices` contains `x` and `y` values, which give the position of the rectangle corner inside the picture (`x` = number of pixels from left edge, `y` = number of pixels from top of image.)

上にプリントした結果では、最初は`bounding_poly`という名前のフィールドがあります。検出された顔を囲む長方形の位置情報が入っています。四角形の各角に一つずつ、`vertices`という名前の4つのサブフィールドがあります。`vertices`（角）の順番は、左上、右上、右下、左下です。 それぞれの `vertices`には`x`と`y`の値があって、それは長方形の角の画像内の位置を表します。（`x`は左端からのピクセル数、`y`は画像の上からのピクセル数）

In order to better visualize the result let's plot the image again, with rectangles around all the detected faces. We will use the `bounding_poly` field in the result. 
<br>
The function `rectangle` from the OpenCV library can be used to plot a rectangle inside an image. It needs the coordinates of the top-left and bottom-right corner, which we can get from the first and third `vertices` of each detected face.

結果を視覚化するために、画像を表示して、そしてその上に検出された顔を囲む長方形を描きます。長方形の情報は`bounding_poly`フィールドから取ります。
<br>
OpenCVライブラリの関数`rectangle`は、画像の中に矩形を描きます。左上隅と右下隅の座標が必要ですが、その情報はの顔の1番目と3番目の`vertices`に入っています。

In [ ]:
Ic = I.copy()
for index,face in enumerate(faces):
    pt1 = (face.bounding_poly.vertices[0].x, face.bounding_poly.vertices[0].y)
    pt2 = (face.bounding_poly.vertices[2].x, face.bounding_poly.vertices[2].y)
    print("Face {} coordinates: ".format(index),pt1, pt2)
    cv2.rectangle(Ic, pt1, pt2, (0,255,0), 5) ## last 2 parameters are for color (R,G,B) and thickness of line
plt.imshow(Ic)
_=plt.axis("off")

By the way, based on the printed (x,y) coordinates of the detected faces, can you tell which belongs to the women on the left side?

ちなみに、上に検出された顔の(x,y)座標をプリントしました。それを見て、何番目の顔が写真の左側にいる女性の顔か、分けりますか？

## Try  it yourself ! / 自分で試そう！

The code above was a basic example. Now you can test the face detection on your own images.
<br>
[Click here](session1-playground.ipynb) to open the notebook with exercise tasks.

上記のコードは基本的な例です。 次は自身の画像で顔検出を試しましょう。
<br>
練習問題のノートブックを開くには[ここをクリックしてください](session1-playground.ipynb)。